___
# mu7RON Demo 3
---

In this notebook we will see how to generate music from an RNN that we have trained

In [1]:
import os
import pickle

import numpy as np
import midi

from mu7ron import maps, temporal, coders, utils

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


You have to use the same parameters that you used when you processed the input data and trained the RNN

In [2]:
off_mode     = False   # whether to encode NoteOffEvents or just NoteOnEvents with 0 velocity
q            = 8       # quantization factor of velocity
q_map        = maps.create_q_map(128, q, encode=True, decode=True)
t            = 8     # the smallest timestep in milliseconds
n_vel        = utils.dynamic_order(128, q) # No. of different available levels of velocity
n_time       = 144   # Available timesteps
n_pitch      = 128 * 2 if off_mode else 128 # + Available pitches
n_pulse      = 0     # Number of added pulses
n_vocab      = n_time + n_pitch + n_vel + n_pulse # Available choices
n_sample     = 24    # batch size
n_input      = 120   # time steps - the temporal dimension
n_output     = 1     # number of timesteps to predict into the future
n_teach      = 12
n_step       = n_sample
n_example    = 1
buffer       = 150
random_state = 117   # "Wake Me... When You Need Me."

time_encoder = temporal.timeslips_encoder #temporal.base_digits_encoder #temporal.timeslips_encoder
time_decoder = temporal.timeslips_decoder # temporal.base_digits_decoder #temporal.timeslips_decoder
ekwa         = dict(t=t, n_time=n_time) # dict(b=n_time) #dict(t=t, n_time=n_time)
dkwa         = dict(t=t) # dict(b=n_time) #dict(t=t)
print('n_vocab: ', n_vocab)

n_vocab:  288


Idealy you should have saved these; a good idea is to pickle them and load them up like this:

In [6]:
WORKING_DIR = os.path.join('mu7ron', 'data', 'models', 'model4', 'params_train_valid.pkl')

with open(WORKING_DIR, 'rb') as file:
    params, train, valid = pickle.load(file)
    for name, value in params.items():
        if name.startswith('time_'):
            value = f"temporal.{str(value).split(' ')[1]}"
        exec(f'{name} = {value}')

Here I have saved my training data as a 3-tuple of parameters, training and validation.

filter out sequences that are too short

In [90]:
valid = [sequence for sequence in train if len(sequence) >= n_input]

We will select a random sample from the validation set to act as a seed.

In [91]:
seed = valid[np.random.randint(len(valid))]

You can listen to the seed below.

In [92]:
utils.play(
    coders.decategorize_output_with_drums(
        seed,
        q=q,
        q_map=q_map,
        n_time=n_time,
        off_mode=off_mode,
        drm_mode=drm_mode,
        time_decoder=time_decoder,
        dkwa=dkwa))

We only need an n_input length section of the seed.

In [93]:
n = np.random.randint(len(seed) - n_input)
one_hot_seed = np.zeros((n_input, n_vocab))
one_hot_seed[range(n_input), seed[n :n + n_input]] = 1.
one_hot_seed.shape

(240, 510)

We are ready to generate some music. We just need to know the path of a pretrained model.  

Here we are going to use beam search: https://en.wikipedia.org/wiki/Beam_search  

A values of 5-10 are quite normal for the beam_width. As the beam_width is increased the algorithm becomes
quite computationally heavy so it is worth keeping it around these values. Plus my implementation, at the moment, will fill up all available memory if you set it to generate a long enough sequence. It is worth taking this into account.

`coders.beam_search` will return a `beam` object which is a python list containing 2-element lists where the first element is a probability and element 2 is the one_hot encoded sequence we have generated.

The last element in `beam` (beam[-1]) will contain the most likely sequence. The sequences still require converting into valid midi sequences.

In [97]:
PATH_TO_MODEL = os.path.join('mu7ron', 'data', 'models', 'model4', 'model_0_5604430437088013_13_Oct_2020_08-38-40.h5')

beam_width = 10
song_length = 1000

beam = coders.beam_search(one_hot_seed, beam_width, path_to_model=PATH_TO_MODEL, n_iter=song_length)

100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [01:45<00:00,  9.46it/s]


In [64]:
beam[:4]

[[0.0,
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]])],
 [0.018919766,
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]])],
 [0.047428966,
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]])],
 [0.047429442,
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 1.],
   

Now listen to the generated music!

In [99]:
my_song = coders.decategorize_output_with_drums(np.argmax(beam[-1][1][n_input:], axis=1), 
                    q=q,
                    q_map=q_map,
                    n_time=n_time,
                    off_mode=off_mode,
                    drm_mode=drm_mode,
                    time_decoder=time_decoder, 
                    dkwa=dkwa)
utils.play(my_song)

Some midi players may need additional midi.events objects added in order to play the sequence but for a bare minimum we should at the very least add a midi.EndOfTrackEvent().

In [41]:
my_song = edit.finalize_midi_sequence(my_song)

Now lets save the song for safe keeping :)

In [42]:
NAME_OF_SONG  = 'my_song.mid'
WHERE_TO_SAVE = os.path.join(os.getcwd(), NAME_OF_SONG)

midi.write_midifile(WHERE_TO_SAVE, my_song)